# Join data files

In [4]:
import pandas as pd

# List of CSV file paths
csv_files = ['ia-data_06-23_08-23.csv', 'ia-data_09-23_12-23.csv', 'ia-data_01-24_03-24.csv', 'ia-data_04-24_06-24.csv', 'ia-data_07-24_11-24.csv']

# Output file
output_file = 'raw_ia_data.csv'

# Create the output file with the header from the first CSV
with open(output_file, 'w') as f_out:
    pd.read_csv(csv_files[0], skip_blank_lines=True).to_csv(f_out, index=False, lineterminator='\n')

# Append the remaining files without the header
for file in csv_files[1:]:
    with open(output_file, 'a') as f_out:
        pd.read_csv(file, skip_blank_lines=True).to_csv(f_out, index=False, header=False, lineterminator='\n')

print("CSV files appended successfully into 'merged.csv'")


C:\Users\Dario\AppData\Local\Temp\ipykernel_9632\2468897074.py:11: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  pd.read_csv(csv_files[0], skip_blank_lines=True).to_csv(f_out, index=False, lineterminator='\n')
C:\Users\Dario\AppData\Local\Temp\ipykernel_9632\2468897074.py:16: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  pd.read_csv(file, skip_blank_lines=True).to_csv(f_out, index=False, header=False, lineterminator='\n')
C:\Users\Dario\AppData\Local\Temp\ipykernel_9632\2468897074.py:16: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  pd.read_csv(file, skip_blank_lines=True).to_csv(f_out, index=False, header=False, lineterminator='\n')
C:\Users\Dario\AppData\Local\Temp\ipykernel_9632\2468897074.py:16: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  pd.read_csv(file, sk

CSV files appended successfully into 'merged.csv'


In [2]:
import pandas as pd
df = pd.read_csv('raw_ia_data.csv')
df['Time'] = pd.to_datetime(df['Time'])
df.sort_values(by=['deviceId', 'Time'], inplace=True)
df.to_csv('raw_ia_data.csv', index=False)

C:\Users\Dario\AppData\Local\Temp\ipykernel_14356\3081564111.py:2: DtypeWarning: Columns (2,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('raw_ia_data.csv')


# Get packets previous to a failure

In [22]:
import pandas as pd
from tqdm import tqdm

ALARM_PUMP = 3
ALARM_EQUIPMENT = 5

df = pd.read_csv('raw_ia_data.csv')
df['Time'] = pd.to_datetime(df['Time'])
df.sort_values(by=['deviceId', 'Time'], inplace=True)
print('START', df.shape[0])

rows_to_drop = []  # Indices to drop
general = {
    'times_to_fail': [], # Time to append
    'interval_groups': [] # Groups to append
}
current = {
    'times_to_fail': [], # Time to append
    'interval_groups': [], # Groups to append,
    'current_rows': []
}
last_row = None
current_group = 1
device_group = ''
for index in tqdm(df.iloc[::-1].index):  # Reverse order
    current_state = df.loc[index, 'state']
    current_time = df.loc[index, 'Time']
    current_device = df.loc[index, 'deviceId']

    if current_state == ALARM_PUMP or current_state == ALARM_EQUIPMENT:    # Apply condition
        if last_row is not None:
            rows_to_drop += current['current_rows'] # Remove invalid interval rows
        current['times_to_fail'] = [0]
        current['interval_groups'] = [current_group]
        current['current_rows'] = [index]
        device_group = current_device
        last_row = {'Time': current_time, 'State': current_state, 'Index': index}
    elif last_row is not None:
        time_diff = last_row['Time'] - current_time
        if  time_diff.total_seconds() > 6 * 3600 or current_device != device_group:   # More than 6 hours
            # End interval
            last_row = None
            rows_to_drop.append(index)  # Mark the row for deletion
            general['times_to_fail'] = current['times_to_fail'] + general['times_to_fail']
            general['interval_groups'] = current['interval_groups'] + general['interval_groups']
            current = {
                'times_to_fail': [],
                'interval_groups': [],
                'current_rows': []
            }
            current_group += 1
        else:
            # Append to interval
            current['times_to_fail'].insert(0, time_diff.total_seconds())
            current['interval_groups'].insert(0, current_group)
            current['current_rows'].insert(0, index)
    else:
      rows_to_drop.append(index)  # Mark the row for deletion

# Drop rows in a single operation (more efficient)
df.drop(rows_to_drop, inplace=True)
# Reset the index after dropping rows (optional)
df.reset_index(drop=True, inplace=True)
df['timeToFail'] = general['times_to_fail']
df['intervalGroup'] = general['interval_groups']

df.to_csv('ia_data_events.csv', index=False)

df.describe()

C:\Users\Dario\AppData\Local\Temp\ipykernel_14356\1574695744.py:7: DtypeWarning: Columns (2,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('raw_ia_data.csv')


START 3162739


100%|██████████| 3162739/3162739 [02:04<00:00, 25350.34it/s]


,Time,angle,distanceSpeed,flow,latitude,longitude,pressure,rawSpeed,speed,state,timeToFail,intervalGroup
count,50917,50917.000000,50917.000000,50917.000000,50917.000000,50917.000000,50917.000000,50917.000000,50917.000000,50917.000000,50917.000000,50917.000000
mean,2024-01-28 01:38:06.128954880,175.404899,25.869387,197.782075,-33.672994,-63.363541,1.430019,0.013970,0.128152,5.484985,10211.029538,1332.809219
min,2023-06-02 00:14:37,0.000000,-47.000000,43.200000,-40.740237,-112.048533,0.000000,-141.000000,0.000000,0.000000,0.000000,1.000000
25%,2023-11-24 12:18:55,86.900000,0.216000,110.000000,-37.768876,-65.683201,0.600000,-0.067000,0.019900,6.000000,4407.000000,672.000000
50%,2024-01-17 14:59:57,176.000000,0.676000,166.000000,-34.446946,-62.459642,1.300000,0.004560,0.087400,6.000000,10096.000000,1325.000000
75%,2024-02-19 20:18:07,261.000000,1.210000,250.000000,-31.993509,-60.204861,2.100000,0.100000,0.167000,6.000000,15738.000000,1997.000000
max,2024-11-20 15:26:18,360.000000,432111.000000,900.000000,31.029446,-57.561363,10.000000,11.500000,141.000000,8.000000,21600.000000,2658.000000
std,NaN,103.259145,2658.519872,109.305323,6.132737,4.852453,1.061016,0.682833,0.670864,1.474771,6425.857476,763.236088


# Get random intervals with NO failures

In [27]:
import numpy as np
from tqdm import tqdm

CASES_REQUIRED = 2658
ALARM_PUMP = 3
ALARM_EQUIPMENT = 5
HOURS_REQUIRED = 6
MIN_PACKETS = 18 # If there is a packet each 15 minutes, in 6 hours should be 24

df = pd.read_csv('raw_ia_data.csv')
df['Time'] = pd.to_datetime(df['Time'])

selected_rows = []
interval_groups = []
intervals_num = 1
pbar = tqdm(total=CASES_REQUIRED)
while intervals_num <= CASES_REQUIRED:
  rand_ind = np.random.randint(0, df.shape[0] - 30) # Padding of 30 last lines
  init_time = df.loc[rand_ind, 'Time']
  init_device = df.loc[rand_ind, 'deviceId']

  curr_ind = rand_ind
  go_on = True
  current_rows = []
  while go_on: # Fix
    row = df.loc[curr_ind]
    time_diff = row['Time'] - init_time

    if row['state'] == ALARM_PUMP or row['state'] == ALARM_EQUIPMENT or row['deviceId'] != init_device:
      # Do not keep the interval
      go_on = False 
    elif time_diff.total_seconds() > HOURS_REQUIRED * 3600:
      # Keep interval
      if len(current_rows) > MIN_PACKETS:
        selected_rows += current_rows
        interval_groups += [intervals_num] * len(current_rows)
        intervals_num += 1
        pbar.update(1)
      go_on = False
    else:
      current_rows.append(row)
      curr_ind += 1

pbar.close()

new_df = pd.DataFrame(selected_rows, columns=df.columns)
new_df['timeToFail'] = [-1] * new_df.shape[0]
new_df['intervalGroup'] = interval_groups
new_df.to_csv('ia_data_no_fails.csv', index=False)
    



C:\Users\Dario\AppData\Local\Temp\ipykernel_14356\722289066.py:10: DtypeWarning: Columns (2,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('raw_ia_data.csv')
100%|██████████| 2658/2658 [00:08<00:00, 306.38it/s]
